In [ ]:
#판다스 넘파이
import pandas as pd
import numpy as np 

#시각화
import seaborn as sns 
import matplotlib.pyplot as plt

#머신러닝
import lightgbm as lgb
from lightgbm import LGBMRegressor

#검증평가. 교차검증 점수. 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error

#기타 (tqdm-> for문보다 빠름)
from tqdm import tqdm
import warnings ; warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('train.csv') #파일이 py와 같은 폴더 내에 있을 경우. 
train.head()

In [ ]:
#컬럼에 해당되는 것만 뽑아내기
train_dst = train.filter(regex = '_dst$', axis='columns')
train_dst

In [ ]:
np.arange(35) *10 + 650 #배열로 만들어 진것을 리스트로 바꾸어 보자 

In [ ]:
list(np.arange(35) *10 + 650)

In [ ]:
#dst값을 치환하기 
train_dst = train.filter(regex = '_dst', axis = 'columns')
train_dst.columns = list(np.arange(35) *10 + 650)
train_dst.columns

In [ ]:
train.filter(regex='_dst$',axis="columns").columns

In [ ]:
train_dst_i = train_dst.interpolate(method = 'linear', axis=1)
train_dst_i.columns

In [ ]:
#spline, polynomial 사용시 interpolate 괄호안에 order = 숫자 를 넣어야함. 
#ex) interpolate(method = 'polynomial', axis=1, order =3)
train_dst_i = train_dst.interpolate(method = 'linear', axis=1) #linear 보간을 완료 , linear이나 쓰고 싶은것으로. 
#대신 곡선화는 order추가해주어야함. axis뒤에 order
train_dst_i.columns = train.filter(regex='_dst$',axis="columns").columns #숫자 컬럼에서 dst 문자컬럼으로 변경 
train_dst_i.columns

In [ ]:
test = pd.read_csv('test.csv')
test

In [ ]:
#test파일을 보간하여 보자. 
test_dst = test.filter(regex='_dst$', axis="columns")
test_dst_i = test_dst.interpolate(method = 'linear', axis=1) #linear 보간을 완료 , linear이나 쓰고 싶은것으로. 
test_dst_i.columns = test.filter(regex='_dst$',axis="columns").columns #숫자 컬럼에서 dst 문자컬럼으로 변경 
test_dst_i.columns

In [ ]:
#데이터 나라시 . 평탄화 작업. 그전 값에서 받아서 채워넣기.

# for i in range() => f{i}_dst -> 문자열 안에 함수를 넣기 
# 34부터 시작하는 이유 980부터 시작하려고 
# 990에 있는 값을 비어있으면 980에 채워넣어라. 
for i in range(34):  
    train_dst_i.loc[train_dst_i[f'{980-(i*10)}_dst'].isna(),
                    f'{980-(i*10)}_dst'] = train_dst_i.loc[train_dst_i[f'{980-(i*10)}_dst'].isna(),
                                                           f'{990-(i*10)}_dst']
    test_dst_i.loc[test_dst_i[f'{980-(i*10)}_dst'].isna(),
                   f'{980-(i*10)}_dst'] = test_dst_i.loc[test_dst_i[f'{980-(i*10)}_dst'].isna(),
                                                         f'{990-(i*10)}_dst']

In [ ]:
#변형된 값을 원본에 업데이트 하기
train.update(train_dst_i)
train

In [ ]:
test.update(test_dst_i)
test

In [ ]:
#램버트 비어 법칙을 적용한 New columns을 만들어 보자.
#입실론 * c 를 ratio컬럼으로 

for i in range(35):
    train[f'{650+(i*10)}_ratio'] = np.log(train[f'{650+(i*10)}_src']/train[f'{650+(i*10)}_dst']) / (0.1*train['rho'])
    test[f'{650+(i*10)}_ratio'] = np.log(test[f'{650+(i*10)}_src']/test[f'{650+(i*10)}_dst']) / (0.1*test['rho'])

In [ ]:
#- + inf 값을 빈값으로 변환 시켜보자

train_r = train.filter(regex='_ratio$',axis='columns').replace(float('-inf'),np.nan).replace(float('inf'),np.nan)
test_r = test.filter(regex='_ratio$',axis='columns').replace(float('-inf'),np.nan).replace(float('inf'),np.nan)

In [ ]:
#보간 하기 전 컬럼 명을 숫자로 바꾸어 주자
train_r.columns = list(650 + 10*np.arange(35))
test_r.columns = list(650 + 10*np.arange(35))

print(train_r.columns)
print(test_r.columns)

In [ ]:
#컬럼을 보간
train_ratio = train_r.interpolate(method='linear',axis=1)
test_ratio = test_r.interpolate(method='linear',axis=1)

#컬럼명을 다시 원래대로 변경 해서 ratio로 변환.
train_ratio.columns = train.filter(regex='_ratio$', axis='columns').columns
test_ratio.columns = test.filter(regex='_ratio$', axis='columns').columns

#ratio로 새로 만들어 졌는지 확인해보자 . 
print(train_ratio.columns)
print(test_ratio.columns)

In [ ]:
# 빈값을 이전 컬럼에서 가져와 채우는 단계
for i in range(34):  
    train_ratio.loc[train_ratio[f'{980-(i*10)}_ratio'].isna(),
                    f'{980-(i*10)}_ratio'] = train_ratio.loc[train_ratio[f'{980-(i*10)}_ratio'].isna(),
                                                           f'{990-(i*10)}_ratio']
    test_ratio.loc[test_ratio[f'{980-(i*10)}_ratio'].isna(),
                   f'{980-(i*10)}_ratio'] = test_ratio.loc[test_ratio[f'{980-(i*10)}_ratio'].isna(),
                                                         f'{990-(i*10)}_ratio']

In [ ]:
#잘 변경된것을 확인하였다면 원본파일을 업데이트 하여주자. 
train.update(train_ratio)
test.update(test_ratio)

In [ ]:
# 변경된 빛 계산 식에 따라 새로운 식을 만들어 보자. 
# {[μ * DPF * d] + G} - {[μ * DPF * d] - G} / 지오메트리 변수를 제거해 준다면.. 가시성이 개선되지 않을까?

for i in range(35):
    train[f'{650 + (10 * i)}_ratio_diff'] = train[f'{650 + (10 * i)}_ratio'] - train['990_ratio']
    test[f'{650 + (10 * i)}_ratio_diff'] = test[f'{650 + (10 * i)}_ratio'] - test['990_ratio']

#상관도가 너무 높은 값을 제외해주고 전체 값이 상관도가 유의미한 값을 가지도록     
for i in range(35):
    train[f'{650 + (10 * i)}_ratio_diff2'] = train[f'{650 + (10 * i)}_ratio'] - (train['850_ratio']+train['860_ratio'])/2
    test[f'{650 + (10 * i)}_ratio_diff2'] = test[f'{650 + (10 * i)}_ratio'] - (test['850_ratio']+test['860_ratio'])/2  

In [ ]:
train_dst = train.filter(regex='_dst$', axis='columns')
test_dst = test.filter(regex='_dst$', axis='columns')

In [ ]:
train_dst.index #(id 값)

In [ ]:
train_dst.loc[0]

In [ ]:
# 평균 대비 dst 컬럼 값의 비율을 새로운 컬럼으로 만들어보자!.
# 위에서 했던 방식대로 하면 길어지니... 새로운 방법으로 해보자구! 

train_dst = train.filter(regex='_dst$', axis='columns')
test_dst = test.filter(regex='_dst$', axis='columns')

#tqdm을 써보자! / 0값은 빈값을 넣어주어 평균화에서 배제하게끔 만들어 준다. 실제 컬럼에는 영향을 주지 않는다. 
for i in tqdm(train_dst.index):
    train_dst.loc[i] = train_dst.loc[i] / train_dst.loc[i].replace(0, np.nan).mean()
   #test를 써야하는데 test는 10000부터 시작이니까... 따로 작성한다. 
for i in tqdm(test_dst.index):
    test_dst.loc[i] = test_dst.loc[i] / test_dst.loc[i].replace(0, np.nan).mean()
    
    
#업데이트 방식이아니라 새로운 컬럼을 만들어 주자. 
for i in range(35):
    train[f'{650 + 10*i}_dst_r'] = train_dst[f'{650 + 10*i}_dst']
    test[f'{650 + 10*i}_dst_r'] = test_dst[f'{650 + 10*i}_dst']